# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Student 1 - __[antoine.bedaton@ulb.be](mailto:antoine.bedaton@ulb.be) - Student ID 459482__
### Student 2 - __[pierre.defraene@ulb.be](mailto:pierre.defraene@ulb.be) - Student ID 463941__
### Student 3 - __[nathan.marotte@ulb.be](mailto:nathan.marotte@ulb.be) - Student ID 459274__

### Video presentation: www.youtube.com/abcd1234

## Project Title


# Introduction

Using data from Taarifa and the Tanzanian Ministry of Water, we are asked to predict which pump are functional, non functional, or functional but need some repairs. The data contains information about each pump (position, region name, population, type of payment, installator, etc ...) and comes in 3 files : 

- training_set_labels : Contains the list of all id followed by their status (functionning, non functionning or needing repairs)

- training_set_values : Contains all the information about each pump with their id that correspond to training_set_labels

- test_set_values : The same structure as training_set_values but for which the status is unknown and that we will have to predict.

First, we will preprocess the data to remove redundent or useless information. For exemple the name of the pump isn't really relevant for guessing if it is working or not, while the name of the constructor is.

Once we reduced the size of the input space, we will run our 3 models on the data, that is training_set_values with their status_group column added.

Our group chose the 3 following models : 

- Decision Tree (rpart library)
- Neural Network (nnet library)
- Random Forest (randomForest library)

Uncomment and execute the following cell to install all the require libraries

In [ ]:
# install.packages('dummies')
# install.packages('rpart')
# install.packages('rpart.plot')
# install.packages('nnet')
# install.packages('randomForest')


We will be using R in version 3.6.1
This can be checked by executing the following cell

In [ ]:
R.version$version.string

We will use the data provided by the competition "Pump it Up: Data Mining the Water Table" available [here](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/])

In [ ]:
training_set_labels <- read.csv(file = 'data/training_set_labels.csv', stringsAsFactors = T)
test_set_values <- read.csv(file = 'data/test_set_values.csv', stringsAsFactors = T)
training_set_values <- read.csv(file = 'data/training_set_values.csv', stringsAsFactors = T)

#training_set_labels
#summary(test_set_values)

# Data preprocessing

## The raw Data
Here is the data as received, as we can see it contains a lot of column and rows, some of which might not be useful

In [ ]:
training_set_values

## The Preprocessing

Before starting the modeling, we must first process the data. We are given a huge number of columns and rows, some of which are not useful to determine the functioning of the pump. For instance, maybe the name of the pump is not important to guess if it is working or not.

### Removing useless columns
First we will check all the columns and check if, by chance, one column has always the same value. If there exist such a column, we can safely delete it since it doesn't bring any information to the model.

In [ ]:
for (colname in names(training_set_values)) {
    list_of_unique <- unique(training_set_values[colname])
    if (nrow(list_of_unique) == 1){
        print(paste("the column", colname, "has always the same value"))
    }
}

One column was found that is always the same value, we will then remove it from the dataset

In [ ]:
training_set_values <- training_set_values[,setdiff(colnames(training_set_values),"recorded_by")] # remove recorded_by train_data

test_set_values <- test_set_values[,setdiff(colnames(test_set_values),"recorded_by")] # remove recorded_by _train_test

Now we can look at the data in a less strict manner. What if for only a few of the entries, we have a different value ? This won't be detected with the above loop, but can be detected manually by checking the number of occurence of the most common value. If it appears, let's say more than 99 % of the time, we assume it is always that value and remove that column.

In [ ]:

# Computes the ratio of the value that appear the most compared to the rest of the data
maxima <- c()
for (colname in names(training_set_values)) {
   maxima <- rbind(maxima, max(table(training_set_values[colname]))/59400)
}

# Shows it nicely
df <- data.frame(names(training_set_values), maxima)
df <- df[order(df$maxima, decreasing=TRUE),]
head(df)


We see that num_private has almost 99% of the time the same value. Given the huge amount of columns and data, we do not feel that it is necessary to keep that information, it would only make the model more complicated for a slight change in the error rate. Therefore we delete that column.

In [ ]:
training_set_values <- training_set_values[,setdiff(colnames(training_set_values),"num_private")] # remove num_private train_set

test_set_values <- test_set_values[,setdiff(colnames(test_set_values),"num_private")] # remove num_private test_est

## Looking at the corelation between columns

When looking through the data, we found multiple columns that were repeated. We suspected that `quantity` and `quantity_group` were highly corolated, so we ran the following command to check directly if there is a bijection for every variable.

In [ ]:
table(training_set_values$quantity, training_set_values$quantity_group)

It appears that when `quantity` is `dry`, 100% of the time `quantity_group` is also dry. Those two columns have a corelation of 100%, it means one of the two is redundant and can safely be removed.

We could then do that for all the combination of columns, but this isn't efficient and we can decide by ourselves by looking at what the columns represent, and test for corelation between them. This can be done via `chiqs.test` of the contingency table of the two columns. 

For exemple with `quantity` and `quantity_group` compared to `quantity` and `payment_type`

In [ ]:
chisq.test(table(training_set_values$quantity, training_set_values$quantity_group))
chisq.test(table(training_set_values$quantity, training_set_values$payment_type))

We get a high X-squared value, this means the two columns are highly correlated. We also check with two uncorrelated columns to see that we indeed get a small X-squared value.

We have to be careful with Chi-squared test since the degree of freedom also influences X-squared value. If we have a high degree of freedom, we might have more problems interpreting the X-square value because of that.

We now have a way to check efficiently if two columns are correlated, so we will use that with the one we suspect are. We can also use the contingency table if we are not sure that they are.

In [ ]:
head(training_set_values)

chisq.test(table(training_set_values$funder, training_set_values$payment_type))
# table(training_set_values$funder, training_set_values$payment_type)  # We see that it's mostly random
chisq.test(table(training_set_values$water_quality, training_set_values$quality_group))


## Feature engineering

For this part, we will check our model and see if we cannot create new features for the data. For exemple we could merge `pay monthly` and `pay annualy` into `periodic_payment`.
However we didn't find that very useful for the data presented.

## Feature selection
We want to look at the correlation between every input and the output. to do so we will make a `chiqs.test` for every column.

In [ ]:
test_correlation <- cbind(training_set_values,training_set_labels[2])

chisq.test(table(test_correlation$amount_tsh, test_correlation$status_group))
chisq.test(table(test_correlation$date_recorded, test_correlation$status_group))
chisq.test(table(test_correlation$funder, test_correlation$status_group))
chisq.test(table(test_correlation$gps_height, test_correlation$status_group))
chisq.test(table(test_correlation$installer, test_correlation$status_group))
chisq.test(table(test_correlation$longitude, test_correlation$status_group))
chisq.test(table(test_correlation$latitude, test_correlation$status_group))
chisq.test(table(test_correlation$wpt_name, test_correlation$status_group))
chisq.test(table(test_correlation$basin, test_correlation$status_group))
chisq.test(table(test_correlation$subvillage, test_correlation$status_group))
chisq.test(table(test_correlation$region, test_correlation$status_group))
chisq.test(table(test_correlation$region_code, test_correlation$status_group))
chisq.test(table(test_correlation$district_code, test_correlation$status_group))
chisq.test(table(test_correlation$lga, test_correlation$status_group))
chisq.test(table(test_correlation$ward, test_correlation$status_group))
chisq.test(table(test_correlation$population, test_correlation$status_group))
chisq.test(table(test_correlation$public_meeting, test_correlation$status_group))
chisq.test(table(test_correlation$scheme_management, test_correlation$status_group))
chisq.test(table(test_correlation$scheme_name, test_correlation$status_group))
chisq.test(table(test_correlation$permit, test_correlation$status_group))
chisq.test(table(test_correlation$construction_year, test_correlation$status_group))
chisq.test(table(test_correlation$extraction_type, test_correlation$status_group))
chisq.test(table(test_correlation$management, test_correlation$status_group))
chisq.test(table(test_correlation$payment, test_correlation$status_group))
chisq.test(table(test_correlation$water_quality, test_correlation$status_group))
chisq.test(table(test_correlation$quantity, test_correlation$status_group))
chisq.test(table(test_correlation$source, test_correlation$status_group))
chisq.test(table(test_correlation$waterpoint_type, test_correlation$status_group))

Now, we will look at pairs of columns that seem similar and compare their contingency table. 

The columns mentionned in the next cell are removed because of their contigency table with another column. We removed them if it indicated that they brought the same or less information to the model

In [ ]:
options(repr.matrix.max.rows=10, repr.matrix.max.cols=100) # option pour la taille des prints



# table(training_set_values$waterpoint_type_group, training_set_values$waterpoint_type)  # less information
training_set_values <- training_set_values[,setdiff(colnames(training_set_values),"waterpoint_type_group")]
test_set_values <- test_set_values[,setdiff(colnames(test_set_values),"waterpoint_type_group")]

# table(training_set_values$source_type, training_set_values$source_class)  # less information
training_set_values <- training_set_values[,setdiff(colnames(training_set_values),"source_class")]
test_set_values <- test_set_values[,setdiff(colnames(test_set_values),"source_class")]

# table(training_set_values$source, training_set_values$source_type)  # less information
training_set_values <- training_set_values[,setdiff(colnames(training_set_values),"source_type")]
test_set_values <- test_set_values[,setdiff(colnames(test_set_values),"source_type")]

# table(training_set_values$water_quality, training_set_values$quality_group)  # less information
training_set_values <- training_set_values[,setdiff(colnames(training_set_values),"quality_group")]
test_set_values <- test_set_values[,setdiff(colnames(test_set_values),"quality_group")]

# table(training_set_values$quantity, training_set_values$quantity_group)  # equivalent, we delete column
training_set_values <- training_set_values[,setdiff(colnames(training_set_values),"quantity_group")]
test_set_values <- test_set_values[,setdiff(colnames(test_set_values),"quantity_group")]

# table(training_set_values$extraction_type, training_set_values$extraction_type_group)  # we can delete column
training_set_values <- training_set_values[,setdiff(colnames(training_set_values),"extraction_type_group")]
test_set_values <- test_set_values[,setdiff(colnames(test_set_values),"extraction_type_group")]

# table(training_set_values$extraction_type, training_set_values$extraction_type_class)  # We can delete column
# training_set_values <- training_set_values[,setdiff(colnames(training_set_values),"extraction_type_class")]

# table(training_set_values$payment, training_set_values$payment_type)  # equivalent,  we delete column
training_set_values <- training_set_values[,setdiff(colnames(training_set_values),"payment_type")]
test_set_values <- test_set_values[,setdiff(colnames(test_set_values),"payment_type")]

# table(training_set_values$management, training_set_values$management_group)  # we delete column
training_set_values <- training_set_values[,setdiff(colnames(training_set_values),"management_group")]
test_set_values <- test_set_values[,setdiff(colnames(test_set_values),"management_group")]

# table(training_set_values$region, training_set_values$region_code)  # very correlated but some cities/code are ambigius
# table(training_set_values$basin, training_set_values$subvillage)  # not really correlated, keep


# TODO REMOVE PRINT

# head(training_set_values)
names(training_set_values)
length(names(training_set_values))
#training_set_values[-c(40, 38, 37, 33, 35, 26, 27, 31, 29, 14, 12)]
training_set_values
# subvillage > region 
training_set_labels


## Missing value imputation
In this part, we will have to replace missing value by something else. One approach could be to simply remove rows that are missing a value if they are not so common, otherwise we can also take the average of that column, or replace the missing value with the most common value. 

We chose to replace the empty value with a new category : `unknown`. This might be a  problem for the model since for exemple, it could indicate that all rows which `funder` is `unkown` are broken, but this could actually be truthful since, indeed, if the funder is unknown, maybe the pump has more chance of breaking

This will be done for all the columns that contain text entries, since there isn't missing data for numbers

In [ ]:
#head(training_set_values, 10)[3, 2]
empty_check <- c("funder", "installer", "basin", "subvillage", "region", "lga", "ward", "scheme_management", "scheme_name", "permit", "extraction_type", "extraction_type_class", "payment", "water_quality", "quantity", "source", "waterpoint_type", "permit", "public_meeting") 
for (row in 1:nrow(head(training_set_values, 10))){
    #print(training_set_values[row])
    for (col in 1:ncol(head(training_set_values, 10))){
        col_name <- names(training_set_values[row])
        if (col_name %in% empty_check){
            if (is.na(training_set_values[row, col])){
                print(paste("Changing"))
                training_set_values[row, col] = as.factor("unknown")
            }
        }
    }
}
head(training_set_values)

## End of preprocessing

We need to transform also categorical variable with 'one-hot-encoding'. To do so, we will use `dummies` package. To facilitate the work, we will merge the training set and the test set to have all different possibilities for each columns.

In [ ]:
library(dummies)

total_data <- rbind(training_set_values,test_set_values) # merge train and test to make dummies variable

factor_variables <- which(sapply(total_data[1,],class)=="factor") # take only factor variable

data_fact <- total_data[,factor_variables]

only_num_var <- total_data[,-factor_variables]

variables_to_keep <- c("waterpoint_type","quantity","extraction_type") # variable to transform

data_factor_onehot <- dummy.data.frame(data_fact[,variables_to_keep], sep="_")


data_factor_onehot[1:2,]

dim(total_data)
dim(training_set_values)

total_with_dummies <- cbind(only_num_var,data_factor_onehot)

# resplit data
# data_train
data_train <- total_with_dummies[1:59400,]
data_train <- cbind(data_train, training_set_labels[2])

data_train


# data_test
data_test <- total_with_dummies[59401:74250,]


# Model selection

As stated in the introduction, we used the three following models/libraries : 

- Decision Tree (rpart library)
- Neural Network (nnet library)
- Random Forest (randomForest library)




## Model 1 : Decision Tree
We chose to use Decision tree since it is one of the simplest machine learning technique, and also that it can help us build intuition about what makes a pump functional or not. 

The decision tree produced might or might not be relevant to the real case problem, but it will be useful to predict the data of the model

In [ ]:
library(rpart)
library(rpart.plot)

data_train

# example model 

model<- rpart(status_group~., data= data_train, method = 'class') # creation of the model


prp(model) # print tree

predict_unseen <- predict(model,data_test, type = 'class') # prediction

status_group <-predict_unseen

data_predict <- test_set_values




data_predict <- cbind(data_predict[1],status_group)

table(data_predict["status_group"] == "functional")

data_predict

write.csv(data_predict,"decisionTree.csv",row.names = FALSE)

## Model 2 : Neural Network

Given how popular Neural Networks are, we will also use it for the data. However we do not expect enormous result with this model since Neural Network work better for other type of problems

In [ ]:
library(nnet)

data_train

# example model 


nnet_model <- nnet(status_group~., data= data_train, size = 8, decay=5e-4, maxit=200) # what we need to change according of the model

predict_unseen <- predict(nnet_model, data_test, type= 'class')

status_group <-predict_unseen

nnet_data_predict <- cbind(data_predict[1],status_group)

nnet_data_predict

table(nnet_data_predict["status_group"] == "functional")

write.csv(nnet_data_predict,"NeuralNetwork.csv",row.names = FALSE)

In [ ]:
#predict_unseen <- predict(nnet_model,only_num_var_train, type= 'class') # test nathan with know variable

In [ ]:
#results <- table(ifelse(predict_unseen == training_set_labels$status_group, "true", "false"))
#results["true"]/(results["true"]+results["false"])

## Model 3 : Random Forest

We chose this model because it is the continuity of decision tree and we think it is a good way to achieve a correct prediction

In [ ]:
library("randomForest")

rf_model<- randomForest(status_group~., data= data_train) # what we need to change according of the model


rf_predict_unseen <- predict(rf_model,data_test, type = 'class')

rf_predict_unseen

status_group <-rf_predict_unseen

rf_data_predict <- test_set_values


rf_data_predict <- cbind(data_predict[1],status_group)


rf_data_predict

write.csv(rf_data_predict,"randomForest.csv",row.names = FALSE)

In [ ]:
rf_model

In [ ]:
# Caret


# Alternative models


TODO : trouver un modèle qui n'existe pas




# Conclusions

In conclusion, we found that decision tree / random forest works best between our 3 models, and we think it would be even better with more accurate data (for exemple pressure sensors, temperature sensors, etc ... ). 

Neural Network is also be a good candidate for this type of problem because it can create abstraction about data in a smarter way than Nearest Neighbor to understand that geographically close waterpumps might or might not suffer the same problem